To run the job, place the data on HDFS by issuing the command (replace the filename) from within the file directory:

The job should be launched by the following command:


The hjs command submits the job; the -file parameter indicates which files will be distributed to the worker nodes (i.e., the source code); the -mapper and -reducer parameters specify the paths to the mapper and reducer scripts; and the -input and -output paths specify the input file(s) and output file paths for the job.

To get the execution time of the job:

To retrieve the results of the computation, run:

To get all the partial outputs. To get the complete output as one file, run:

## mapper.py

In [7]:
import os
import sys
import math 

for line in sys.stdin:
        entry = line.strip().split(",")
        
        # Pickup Centroid Latitude, Pickup Centroid Longitude, Dropoff Centroid Latitude and Dropoff Centroid Longitude
        pick_la, pick_lon, drop_la, drop_lon = entry[11],entry[12],entry[14],entry[15]
        
        # pick up community area
        area = entry[8]
        
        # calculate euclidean distance for each trip
        dist = math.sqrt((pick_la - drop_la)**2 + (pick_lon - drop_lon)**2)
        
        print('{}\t{}'.format(area,dist))
        

## reducer.py

In [10]:
import os
import sys

current_area = None
current_dist = 0.0
area = None
dist = 0.0

# number of trips in this area
num_trips = 0

for line in sys.stdin:
    area, dist = line.split('\t', 1)
            
    # convert distance (currently a string) to int
    try:
        dist = float(dist)
    except ValueError:
        # dist was not a number, so silently
        # ignore/discard this line
        continue

    # Since Hadoop sorts map output by key (here: area) before it is passed to the reducer
    if current_area == area:
        current_dist += dist
        num_trips += 1
    else:
        if current_area and i != 0:
            # write result to STDOUT
            print('{}\t{}'.format(current_area, current_dist/num_trips))
        current_area = area
        current_dist = dist
        num_trips = 0

# Output the last area if needed
if current_area == area and num_trips !=0:
    print('{}\t{}'.format(current_area, current_dist/num_trips))